# Pre-Season Model Training - All Tracks

This notebook trains lap time prediction and tire degradation models for all tracks and races in the season.
Models are saved to disk for instant loading on race day.

In [1]:
# Force reload modules
import importlib
import sys

modules_to_reload = [mod for mod in sys.modules.keys() if 'toyota_gr_cup_analytics' in mod]
for mod in modules_to_reload:
    del sys.modules[mod]

print(f"Removed {len(modules_to_reload)} modules from cache")

Removed 0 modules from cache


In [2]:
from toyota_gr_cup_analytics.data_processing.loaders import load_lap_data, load_weather_data
from toyota_gr_cup_analytics.models.feature_engineering import engineer_features
from toyota_gr_cup_analytics.models.predictive import TireDegradationModel, LapTimePredictorModel
from toyota_gr_cup_analytics.models.model_persistence import save_model_bundle, list_saved_models
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path

print("✅ Imports complete")

✅ Imports complete


## Configuration

In [ ]:
# 2025 Toyota Gazoo Racing GR Cup North America Season Schedule
SEASON_SCHEDULE = [
    {'track': 'sonoma', 'races': [1, 2], 'date': 'March 28-30, 2025'},      # Sonoma Raceway
    {'track': 'cota', 'races': [1, 2], 'date': 'April 25-27, 2025'},        # Circuit of the Americas
    {'track': 'sebring', 'races': [1, 2], 'date': 'May 16-18, 2025'},       # Sebring International Raceway
    {'track': 'vir', 'races': [1, 2], 'date': 'July 18-20, 2025'},          # VIRginia International Raceway
    {'track': 'road_america', 'races': [1, 2], 'date': 'August 15-17, 2025'},  # Road America
    {'track': 'barber', 'races': [1, 2], 'date': 'September 5-7, 2025'},    # Barber Motorsports Park
    {'track': 'indianapolis', 'races': [1, 2], 'date': 'October 17-19, 2025'},  # Indianapolis Motor Speedway
]

# Model configuration
MODEL_CONFIG = {
    'tire_model_type': 'linear',  # 'linear' or 'gradient_boosting'
    'lap_model_type': 'random_forest',  # 'random_forest' or 'gradient_boosting'
    'n_estimators': 100,
    'test_size': 0.2,
    'random_state': 42
}

print(f"2025 Season schedule: {len(SEASON_SCHEDULE)} tracks")
print(f"Total races to process: {sum(len(t['races']) for t in SEASON_SCHEDULE)}")

## Training Function

In [ ]:
def train_track_models(track: str, race: int, config: dict) -> dict:
    """
    Train models for a specific track and race.
    
    Returns:
        Dictionary with models and evaluation metrics
    """
    print(f"\n{'='*60}")
    print(f"Training: {track.upper()} - Race {race}")
    print(f"{'='*60}")
    
    try:
        # Load data
        print("\n1. Loading data...")
        lap_data = load_lap_data(track, race)
        weather_data = load_weather_data(track, race)
        
        if lap_data.empty:
            print(f"  ⚠️  No lap data found for {track} Race {race}")
            return None
        
        print(f"  ✓ Loaded {len(lap_data)} lap records, {len(weather_data)} weather records")
        
        # Engineer features (IMPORTANT: pass track parameter for track-specific handling)
        print("\n2. Engineering features...")
        X, y = engineer_features(lap_data, weather_data, track=track)
        
        if len(X) == 0:
            print(f"  ⚠️  Feature engineering produced 0 samples")
            return None
            
        print(f"  ✓ Features: {X.shape}, Target: {y.shape}")
        print(f"  ✓ Mean lap time: {y.mean():.2f}s (min: {y.min():.2f}s, max: {y.max():.2f}s)")
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=config['test_size'], random_state=config['random_state']
        )
        print(f"  ✓ Train: {len(X_train)}, Test: {len(X_test)}")
        
        # Train tire degradation model
        print("\n3. Training Tire Degradation Model...")
        tire_model = TireDegradationModel(degradation_type=config['tire_model_type'])
        tire_model.fit(X_train, y_train)
        tire_metrics = tire_model.evaluate(X_test, y_test)
        print(f"  ✓ MAE: {tire_metrics['mae']:.2f}s | RMSE: {tire_metrics['rmse']:.2f}s | R²: {tire_metrics['r2']:.3f}")
        
        # Train lap time prediction model
        print("\n4. Training Lap Time Prediction Model...")
        lap_model = LapTimePredictorModel(
            n_estimators=config['n_estimators'],
            model_type=config['lap_model_type'],
            random_state=config['random_state']
        )
        lap_model.fit(X_train, y_train)
        lap_metrics = lap_model.evaluate(X_test, y_test)
        print(f"  ✓ MAE: {lap_metrics['mae']:.2f}s | RMSE: {lap_metrics['rmse']:.2f}s | R²: {lap_metrics['r2']:.3f}")
        
        # Show top features
        importances = lap_model.feature_importance()
        top_5 = sorted(importances.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"\n  Top 5 Features:")
        for feat, imp in top_5:
            print(f"    - {feat}: {imp:.3f}")
        
        # Save models
        print("\n5. Saving models...")
        bundle = {
            'tire_model': tire_model,
            'lap_model': lap_model,
            'metadata': {
                'track': track,
                'race': race,
                'training_samples': len(X_train),
                'test_samples': len(X_test),
                'features': list(X.columns),
                'tire_metrics': tire_metrics,
                'lap_metrics': lap_metrics,
                'config': config
            }
        }
        
        bundle_name = f"{track}_race{race}_models"
        save_model_bundle(bundle, bundle_name)
        print(f"  ✓ Saved as: {bundle_name}.pkl")
        
        return {
            'track': track,
            'race': race,
            'tire_metrics': tire_metrics,
            'lap_metrics': lap_metrics,
            'bundle_name': bundle_name,
            'training_samples': len(X_train),
            'test_samples': len(X_test)
        }
        
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

## Train All Models

In [5]:
# Train models for all tracks and races
results = []

for track_info in SEASON_SCHEDULE:
    track = track_info['track']
    races = track_info['races']
    
    for race in races:
        result = train_track_models(track, race, MODEL_CONFIG)
        if result:
            results.append(result)

print(f"\n{'='*60}")
print(f"TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Successfully trained: {len(results)} track/race combinations")

2025-11-10 07:26:54.971 | INFO     | toyota_gr_cup_analytics.data_processing.loaders:load_lap_data:36 - Loading lap data for barber Race 1


2025-11-10 07:26:54.975 | INFO     | toyota_gr_cup_analytics.data_processing.loaders:load_lap_data:98 - Loaded 579 lap records for barber Race 1 from 23_AnalysisEnduranceWithSections_Race 1_Anonymized.CSV


2025-11-10 07:26:54.975 | INFO     | toyota_gr_cup_analytics.data_processing.loaders:load_weather_data:159 - Loading weather data for barber Race 1


2025-11-10 07:26:54.976 | INFO     | toyota_gr_cup_analytics.data_processing.loaders:load_weather_data:197 - Loaded 43 weather records for barber Race 1


2025-11-10 07:26:54.977 | INFO     | toyota_gr_cup_analytics.models.feature_engineering:engineer_features:134 - Engineering features for lap time prediction


...

2025-11-10 07:26:56.325 | INFO     | toyota_gr_cup_analytics.models.predictive:fit:80 - Top 5 features: [('rolling_mean_3', np.float64(0.5394411011593959)), ('rolling_std_3', np.float64(0.36149412745069903)), ('fuel_load_proxy', np.float64(0.038191036188877495)), ('lap_number', np.float64(0.03010347465656443)), ('stint_number', np.float64(0.01810937781576818))]


2025-11-10 07:26:56.328 | INFO     | toyota_gr_cup_analytics.models.predictive:evaluate:180 - Lap time model - MAE: 0.549s, RMSE: 1.312s, R²: 0.797


2025-11-10 07:26:56.331 | INFO     | toyota_gr_cup_analytics.models.model_persistence:save_model_bundle:125 - Saving model bundle with 3 models to /Users/gilesjm/Repo/trd-hackathon/notebooks/models/vir_race2_models.pkl


2025-11-10 07:26:56.340 | INFO     | toyota_gr_cup_analytics.models.model_persistence:save_model_bundle:127 - Bundle saved successfully (1001.6 KB)


  ✓ MAE: 0.55s | RMSE: 1.31s | R²: 0.797

  Top 5 Features:
    - rolling_mean_3: 0.539
    - rolling_std_3: 0.361
    - fuel_load_proxy: 0.038
    - lap_number: 0.030
    - stint_number: 0.018

5. Saving models...
  ✓ Saved as: vir_race2_models.pkl

TRAINING COMPLETE
Successfully trained: 9 track/race combinations


## Results Summary

In [6]:
# Create results summary DataFrame
if results:
    summary_data = []
    for r in results:
        summary_data.append({
            'Track': r['track'].upper(),
            'Race': r['race'],
            'Train Samples': r['training_samples'],
            'Test Samples': r['test_samples'],
            'Tire MAE (s)': f"{r['tire_metrics']['mae']:.2f}",
            'Tire R²': f"{r['tire_metrics']['r2']:.3f}",
            'Lap MAE (s)': f"{r['lap_metrics']['mae']:.2f}",
            'Lap R²': f"{r['lap_metrics']['r2']:.3f}",
            'Model File': r['bundle_name']
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\n" + "="*120)
    print("MODEL PERFORMANCE SUMMARY")
    print("="*120)
    print(summary_df.to_string(index=False))
    
    # Calculate averages
    tire_mae_avg = sum(r['tire_metrics']['mae'] for r in results) / len(results)
    lap_mae_avg = sum(r['lap_metrics']['mae'] for r in results) / len(results)
    tire_r2_avg = sum(r['tire_metrics']['r2'] for r in results) / len(results)
    lap_r2_avg = sum(r['lap_metrics']['r2'] for r in results) / len(results)
    
    print("\n" + "="*120)
    print("SEASON AVERAGES")
    print("="*120)
    print(f"Tire Model:     MAE = {tire_mae_avg:.2f}s | R² = {tire_r2_avg:.3f}")
    print(f"Lap Time Model: MAE = {lap_mae_avg:.2f}s | R² = {lap_r2_avg:.3f}")
else:
    print("\n⚠️  No models were successfully trained")


MODEL PERFORMANCE SUMMARY
  Track  Race  Train Samples  Test Samples Tire MAE (s) Tire R² Lap MAE (s) Lap R²           Model File
 BARBER     1            463           116         3.19   0.341        0.59  0.951  barber_race1_models
 BARBER     2            481           121         5.73   0.651        3.37  0.587  barber_race2_models
   COTA     1            388            98        12.81   0.680        5.23  0.515    cota_race1_models
SEBRING     1            320            81         1.33   0.406        1.63 -0.089 sebring_race1_models
SEBRING     2            265            67         6.24   0.704        0.79  0.983 sebring_race2_models
 SONOMA     1            535           134        13.92  -4.965        4.02 -0.566  sonoma_race1_models
 SONOMA     2            520           130         8.49   0.426        1.88  0.930  sonoma_race2_models
    VIR     1            332            83         9.43   0.432        1.84  0.911     vir_race1_models
    VIR     2            352         

## List All Saved Models

In [7]:
print("\nSaved Model Files:")
print("=" * 60)
saved_models = list_saved_models()
for model_path in sorted(saved_models):
    size_kb = model_path.stat().st_size / 1024
    print(f"  {model_path.name:50s} ({size_kb:6.1f} KB)")

print(f"\nTotal: {len(saved_models)} model files")

2025-11-10 07:26:56.350 | INFO     | toyota_gr_cup_analytics.models.model_persistence:list_saved_models:95 - Found 9 saved models in /Users/gilesjm/Repo/trd-hackathon/notebooks/models



Saved Model Files:
  barber_race1_models.pkl                            (1082.3 KB)
  barber_race2_models.pkl                            ( 968.5 KB)
  cota_race1_models.pkl                              (1115.5 KB)
  sebring_race1_models.pkl                           ( 899.5 KB)
  sebring_race2_models.pkl                           ( 849.1 KB)
  sonoma_race1_models.pkl                            (1304.3 KB)
  sonoma_race2_models.pkl                            (1498.7 KB)
  vir_race1_models.pkl                               (1041.2 KB)
  vir_race2_models.pkl                               (1001.6 KB)

Total: 9 model files


## Test Loading a Model

In [8]:
from toyota_gr_cup_analytics.models.model_persistence import load_model_bundle

if results:
    # Test loading the first trained model
    test_bundle_name = results[0]['bundle_name']
    print(f"Testing model loading: {test_bundle_name}")
    print("=" * 60)
    
    loaded_bundle = load_model_bundle(test_bundle_name)
    
    print(f"\n✓ Successfully loaded bundle")
    print(f"  Keys: {list(loaded_bundle.keys())}")
    print(f"\n  Metadata:")
    for key, value in loaded_bundle['metadata'].items():
        if key not in ['features', 'config', 'tire_metrics', 'lap_metrics']:
            print(f"    {key}: {value}")
    
    print(f"\n  Lap Model Performance (from metadata):")
    lap_meta = loaded_bundle['metadata']['lap_metrics']
    print(f"    MAE: {lap_meta['mae']:.2f}s")
    print(f"    R²: {lap_meta['r2']:.3f}")
    
    print("\n✅ Model loading successful!")

2025-11-10 07:26:56.353 | INFO     | toyota_gr_cup_analytics.models.model_persistence:load_model_bundle:159 - Loading model bundle from /Users/gilesjm/Repo/trd-hackathon/notebooks/models/barber_race1_models.pkl


2025-11-10 07:26:56.360 | INFO     | toyota_gr_cup_analytics.models.model_persistence:load_model_bundle:161 - Loaded bundle with 3 models


Testing model loading: barber_race1_models

✓ Successfully loaded bundle
  Keys: ['tire_model', 'lap_model', 'metadata']

  Metadata:
    track: barber
    race: 1
    training_samples: 463
    test_samples: 116

  Lap Model Performance (from metadata):
    MAE: 0.59s
    R²: 0.951

✅ Model loading successful!


## Usage on Race Day

To use these models on race day:

```python
from toyota_gr_cup_analytics.models.model_persistence import load_model_bundle

# Load models for specific track/race
models = load_model_bundle('barber_race1_models')
lap_model = models['lap_model']
tire_model = models['tire_model']

# Make predictions
next_lap_time, metadata = lap_model.predict_next_lap(
    current_state={'car_id': 5, 'lap_number': 20, ...},
    weather_conditions={'track_temp': 32.0, ...}
)

# Predict stint performance
stint_times, stint_stats = tire_model.predict_stint_performance(
    base_lap_time=98.5,
    stint_length=15,
    car_id=5,
    temperature=32.0
)
```

**Benefits:**
- ⚡ Instant loading (<1 second)
- 🎯 Track-specific predictions
- 📊 Consistent performance across sessions
- 💾 No retraining needed